In [1]:
import pandas as pd
import re
from difflib import get_close_matches
from fuzzywuzzy import process


C:\Users\epoh2\Anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cosmetics1=pd.read_csv(r"C:\Users\epoh2\Documents\NSS\NSS\Python\projects\Cosmetics\data\cosmetics3.csv")

In [3]:
cosmetics1.head(1)

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,13463-67-7,6,Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1


adding titanium dioxide indicator

In [5]:
for ind,row in cosmetics1.iterrows():
    if 'Titanium dioxide' in row.ChemicalName:
        cosmetics1.loc[ind,'Titanium_dioxide']=True
    else : 
        cosmetics1.loc[ind,'Titanium_dioxide']=False

In [6]:
cosmetics1.head(1)

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount,Titanium_dioxide
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,6,Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True


In [7]:
cosmetics1.columns

Index(['CDPHId', 'ProductName', 'CSFId', 'CSF', 'CompanyId', 'CompanyName',
       'BrandName', 'PrimaryCategoryId', 'PrimaryCategory', 'SubCategoryId',
       'SubCategory', 'CasId', 'CasNumber', 'ChemicalId', 'ChemicalName',
       'InitialDateReported', 'MostRecentDateReported', 'DiscontinuedDate',
       'ChemicalCreatedAt', 'ChemicalUpdatedAt', 'ChemicalDateRemoved',
       'ChemicalCount', 'Titanium_dioxide'],
      dtype='object')

dropping columns

In [9]:
cosmetics6=cosmetics1.drop(columns = ['CDPHId', 'CSFId', 'CSF', 'CompanyId', 'PrimaryCategoryId', 'ChemicalId','CasId','CasNumber', 'SubCategoryId'])

In [10]:
cosmetics6.head(1)

,ProductName,CompanyName,BrandName,PrimaryCategory,SubCategory,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount,Titanium_dioxide
0,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,New Avon LLC,AVON,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True


renaming columns

In [12]:
cosmetics_main=cosmetics6.rename(columns={'ProductName':'product', 'CompanyId':'company','BrandName':'brand','PrimaryCategory':'category','SubCategory': 'subcategory','ChemicalName': 'chemical', 'InitialDateReported': 'initial_reported_date','MostRecentDateReported':'most_recent_date_reported','DiscontinuedDate':'discontinued_date','ChemicalCreatedAt':'chemical_created_date','ChemicalUpdatedAt':'chemical_updated_date','ChemicalDateRemoved':'chemical_removed','ChemicalCount':'chemicalcount','Titanium_dioxide':'titanium_dioxide','CompanyName':'company'})

In [13]:
cosmetics_main.to_csv('cosmetics_main.csv')

step one of normalizing columns: lowering column text and dropping unneeded spaces

In [15]:
cosmetics_main['brand']=cosmetics_main['brand'].dropna().str.strip().str.lower()

In [16]:
cosmetics_main['company']=cosmetics_main['company'].dropna().str.strip().str.lower()

In [17]:
cosmetics_main['product']=cosmetics_main['product'].str.lower().str.strip()

In [18]:
cosmetics_main['chemical']=cosmetics_main['chemical'].str.lower().str.strip()

In [19]:
cosmetics_main.head(1)

,product,company,brand,category,subcategory,chemical,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide
0,ultra color rich extra plump lipstick-all shades,new avon llc,avon,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True


In [20]:
cosmetics_main['chemical'].value_counts().head()

chemical
titanium dioxide                                                                                       93480
silica, crystalline (airborne particles of respirable size)                                             2817
retinol/retinyl esters, when in daily dosages in excess of 10,000 iu, or 3,000 retinol equivalents.     2154
mica                                                                                                    1919
butylated hydroxyanisole                                                                                1888
Name: count, dtype: int64

In [21]:
cosmetics_main['chemical'].unique()

array(['titanium dioxide', 'distillates (coal tar)', 'estragole',
       'cocamide diethanolamine', 'toluene',
       'chromium (hexavalent compounds)', 'retinol',
       'retinol/retinyl esters, when in daily dosages in excess of 10,000 iu, or 3,000 retinol equivalents.',
       'vitamin a', 'vitamin a palmitate', 'butylated hydroxyanisole',
       'coffea arabica extract', 'lauramide diethanolamine', 'coffee',
       'silica, crystalline (airborne particles of respirable size)',
       'carbon black (airborne, unbound particles of respirable size)',
       'carbon black', 'genistein (purified)', 'progesterone',
       '2,4-hexadienal (89% trans, trans isomer; 11% cis, trans isomer)',
       'methyleugenol', 'carbon-black extracts', 'retinyl palmitate',
       'o-phenylphenol', 'acrylamide', 'formaldehyde (gas)',
       'ginkgo biloba extract', 'mica', 'ethylene glycol',
       'acetic acid, retinyl ester', 'ethyl acrylate', 'trade secret',
       'methanol', 'mineral oils, untreated 

matching row values for chemicals

In [23]:
for ind,row in cosmetics_main.iterrows():
    chemicall= str(row.chemical)
    if 'titanium dioxide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='titanium dioxide'
    if 'distillates (coal tar)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coal'
    if 'estragole' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='estragole'
    if 'cocamide diethanolamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide diethanolamine'
    if 'toluene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='toluene'
    if 'chromium (hexavalent compounds)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='chromium'
    if 'retinol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinol'
    if 'retinol/retinyl esters, when in daily dosages in excess of 10,000 iu, or 3,000 retinol equivalents.' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinol'
    if 'vitamin a' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='vitamin a'
    if 'vitamin a palmitate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='vitamin a'
    if 'butylated hydroxyanisole' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='butylated hydroxyanisole'
    if 'coffea arabica extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coffee'
    if 'lauramide diethanolamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='lauramide diethanolamine'
    if 'coffee' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coffee'
    if 'silica, crystalline (airborne particles of respirable size)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='silica'
    if 'carbon black (airborne, unbound particles of respirable size)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='carbon black'
    if 'carbon black' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='carbon black'
    if 'genistein (purified)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='genistein'
    if 'progesterone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='progesterone'
    if '2,4-hexadienal (89% trans, trans isomer; 11% cis, trans isomer)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='hexadienal'
    if 'methyleugenol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='methyleaugenol'
    if 'carbon-black extracts' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='carbon black'
    if 'retinyl palmitate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinol'
    if 'o-phenylphenol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='phenylphenol'
    if 'acrylamide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='acrylamide'
    if 'formaldehyde (gas)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='formaldehyde'
    if 'ginkgo biloba extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='ginkgo bilboa extract'
    if 'mica' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='mica'
    if 'ethylene glycol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='ethylene gycol'
    if 'acetic acid retinyl ester' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinyl'
    if 'ethyl acrylate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='ethyl acrylate'
    if 'trade secret' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='trade secret'
    if 'methanol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='methanol'
    if  'mineral oils, untreated and mildly treated' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='mineral oils'
    if 'diethanolamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='diethanolamine'
    if 'tea-lauryl sulfate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='sulfate'
    if 'retinyl acetate' in chemicall: 
        cosmetics_main.loc[ind,'chemicals']='acetate'
    if 'lead acetate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='acetate'
    if 'talc' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='talc'
    if 'triethanolamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='triethanolamine'
    if 'o-phenylenediamine and its salts' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='phenlylenediaine'
    if 'safrole' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='safrole'
    if 'styrene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='stryene'
    if 'acetaldehyde' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='acetaldehyde'
    if 'cocamide dea' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide diethanolamine'
    if '1,4-dioxane' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='dioxane'
    if 'arsenic (inorganic arsenic compounds)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='arsenic'
    if 'dichloroacetic acid' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='dichloroacetic acid'
    if 'ethylene oxide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='ethylene oxide'
    if 'lead' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='lead'
    if 'dichloromethane (methylene chloride)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='dichloromethane'
    if 'benzene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzene'
    if 'benzyl chloride' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzyl chloride'
    if 'n-nitrosodimethylamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='nitrosodimethylamine'
    if 'propylene oxide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='propylene oxide'
    if 'methyl chloride' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='methyl chloride'
    if 'cadmium and cadmium compounds' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cadmium'
    if 'n-methylpyrrolidone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='methylpyrrolidone'
    if 'di-n-butyl phthalate (dbp)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='dibutyl phthalate'
    if 'coal tars' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coal'
    if 'all-trans retinoic acid' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinol'
    if 'quinoline and its strong acid salts' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='quinoline'
    if 'methylene glycol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='methylene glycol'
    if 'benzophenone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzophenone'
    if 'cocamide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide diethanolamin'
    if 'lauramide dea' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='lauramide dea'
    if 'aloe vera, whole leaf extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='aloe'
    if 'musk xylene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='musk xylene'
    if 'aspirin' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='aspirin'
    if 'coal tar' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coal'
    if 'benzophenone-3' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzophenon'
    if 'quartz' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='quartz'
    if 'talc containing asbestiform fibers' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='talc'
    if 'sodium bromate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='sodium'
    if 'phenacetin' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='phenacetic'
    if 'arsenic (inorganic oxides)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='arsenic'
    if 'mercury and mercury compounds' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='mercury'
    if 'p-aminodiphenylamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='aminodiphenylamine'
    if 'permethrin' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='permethrin'
    if 'acetylsalicylic acid' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='salicylic acid'
    if 'coal tar extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coal'
    if 'selenium sulfide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='selenium'
    if 'oil orange ss' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='oil orange ss'
    if 'spironolactone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='spironolactone'
    if 'nickel (metallic)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='nickel'
    if 'caffeic acid' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='caffine'
    if 'cocamide mea' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide mea'
    if 'cosmetic talc' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='talc'
    if 'c.i. acid red 114' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='c.i. acid red 114'
    if 'caffeine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='caffeine'
    if 'benzophenone-4' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzophenone'
    if 'ethanol in alcoholic beverages' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='ethanol'
    if 'formaldehyde' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='formaldehyde'
    if 'cocamide diethanolamine (dea)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide diethanolamine'
    if 'coffee extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coffee'
    if 'retinol palmitate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='retinol'
    if 'coffee bean extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coffee'
    if 'propylene glycol mono-t-butyl ether' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='propylene glycol mono-t-butyl ether'
    if 'avobenzone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='avonbenzone'
    if 'coal tar solution' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coal'
    if 'pulegone' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='pulegone'
    if 'titanium dioxide (airborne, unbound particles of respirable size)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='titanium dioxide'
    if 'beta-myrcene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='beta-myrcene'
    if 'talc (powder)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='talc'
    if '2,2-bis(bromomethyl)-1,3-propanediol' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='bromomethyl'
    if 'benzo[a]pyrene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzo[a]pyrene'
    if 'benz[a]anthracene' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benz[a]anthracene'
    if 'extract of coffee bean' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='coffee'
    if 'goldenseal root powder' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='goldenseal'
    if 'isopropyl alcohol manufacture using strong acids' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='isopropyl alcohol'
    if '2-propyleneacrolein' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='propyleneacrolein'
    if 'n,n-dimethyl-p-toluidine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='n,n-dimethyl-p-toluidin'
    if 'formaldehyde solution' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='formaldehyde'
    if 'n-nitrosodiethanolamine' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='nitrosodiethanolamine'
    if 'benzophenone-2' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='benzophenone'
    if 'vinyl acetate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='acetate'
    if 'trichloroacetic acid' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='trichloroacetic acid'
    if 'phenacemide' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='phenacemide'
    if 'aloe vera, non-decolorized whole leaf extract' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='aloe'
    if 'polygeenan' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='polygeenan'
    if 'diethanolamides of the fatty acids of coconut oil' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='cocamide diethanolamine'
    if 'bisphenol a (bpa)' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='bisphenol'
    if'hydrous magnesium silicate' in chemicall:
        cosmetics_main.loc[ind,'chemicals']='magnesium'
    else:
        cosmetics_main.loc[ind,'chemicals']='N/A'

 above code didnt work so this fixes the problem to create chemicals cleaned

In [25]:
chemical_mapping = {
    'titanium dioxide': 'titanium dioxide',
    'distillates (coal tar)': 'coal',
    'estragole': 'estragole',
    'cocamide diethanolamine': 'cocamide diethanolamine',
    'toluene': 'toluene',
    'chromium (hexavalent compounds)': 'chromium',
    'retinol': 'retinol',
    'retinol/retinyl esters': 'retinol',
    'vitamin a': 'vitamin a',
    'vitamin a palmitate': 'vitamin a',
    'butylated hydroxyanisole': 'butylated hydroxyanisole',
    'coffea arabica extract': 'coffee',
    'lauramide diethanolamine': 'lauramide diethanolamine',
    'coffee': 'coffee',
    'silica, crystalline': 'silica',
    'carbon black': 'carbon black',
    'genistein (purified)': 'genistein',
    'progesterone': 'progesterone',
    '2,4-hexadienal': 'hexadienal',
    'methyleugenol': 'methyleugenol',
    'retinyl palmitate': 'retinol',
    'o-phenylphenol': 'phenylphenol',
    'acrylamide': 'acrylamide',
    'formaldehyde': 'formaldehyde',
    'ginkgo biloba extract': 'ginkgo bilboa extract',
    'mica': 'mica',
    'ethylene glycol': 'ethylene glycol',
    'acetic acid retinyl ester': 'retinyl',
    'ethyl acrylate': 'ethyl acrylate',
    'trade secret': 'trade secret',
    'methanol': 'methanol',
    'mineral oils': 'mineral oils',
    'diethanolamine': 'diethanolamine',
    'tea-lauryl sulfate': 'sulfate',
    'retinyl acetate': 'acetate',
    'lead acetate': 'acetate',
    'talc': 'talc',
    'triethanolamine': 'triethanolamine',
    'o-phenylenediamine': 'phenylenediamine',
    'safrole': 'safrole',
    'styrene': 'styrene',
    'acetaldehyde': 'acetaldehyde',
    'cocamide dea': 'cocamide diethanolamine',
    '1,4-dioxane': 'dioxane',
    'arsenic': 'arsenic',
    'dichloroacetic acid': 'dichloroacetic acid',
    'ethylene oxide': 'ethylene oxide',
    'lead': 'lead',
    'dichloromethane': 'dichloromethane',
    'benzene': 'benzene',
    'benzyl chloride': 'benzyl chloride',
    'n-nitrosodimethylamine': 'nitrosodimethylamine',
    'propylene oxide': 'propylene oxide',
    'methyl chloride': 'methyl chloride',
    'cadmium': 'cadmium',
    'n-methylpyrrolidone': 'methylpyrrolidone',
    'di-n-butyl phthalate': 'dibutyl phthalate',
    'coal tars': 'coal',
    'quinoline': 'quinoline',
    'methylene glycol': 'methylene glycol',
    'benzophenone': 'benzophenone',
    'cocamide': 'cocamide diethanolamine',
    'lauramide dea': 'lauramide dea',
    'aloe vera': 'aloe',
    'musk xylene': 'musk xylene',
    'aspirin': 'aspirin',
    'quartz': 'quartz',
    'sodium bromate': 'sodium',
    'phenacetin': 'phenacetin',
    'mercury': 'mercury',
    'p-aminodiphenylamine': 'aminodiphenylamine',
    'permethrin': 'permethrin',
    'acetylsalicylic acid': 'salicylic acid',
    'selenium sulfide': 'selenium',
    'oil orange ss': 'oil orange ss',
    'spironolactone': 'spironolactone',
    'nickel': 'nickel',
    'caffeic acid': 'caffeine',
    'cocamide mea': 'cocamide mea',
    'c.i. acid red 114': 'c.i. acid red 114',
    'caffeine': 'caffeine',
    'benzophenone-4': 'benzophenone',
    'ethanol in alcoholic beverages': 'ethanol',
    'coffee extract': 'coffee',
    'propylene glycol mono-t-butyl ether': 'propylene glycol mono-t-butyl ether',
    'avobenzone': 'avobenzone',
    'pulegone': 'pulegone',
    'beta-myrcene': 'beta-myrcene',
    '2,2-bis(bromomethyl)-1,3-propanediol': 'bromomethyl',
    'benzo[a]pyrene': 'benzo[a]pyrene',
    'benz[a]anthracene': 'benz[a]anthracene',
    'extract of coffee bean': 'coffee',
    'goldenseal root powder': 'goldenseal',
    'isopropyl alcohol': 'isopropyl alcohol',
    '2-propyleneacrolein': 'propyleneacrolein',
    'n,n-dimethyl-p-toluidine': 'dimethyl-p-toluidine',
    'n-nitrosodiethanolamine': 'nitrosodiethanolamine',
    'benzophenone-2': 'benzophenone',
    'vinyl acetate': 'acetate',
    'trichloroacetic acid': 'trichloroacetic acid',
    'phenacemide': 'phenacemide',
    'polygeenan': 'polygeenan',
    'bisphenol a': 'bisphenol',
    'hydrous magnesium silicate': 'magnesium',
}


def map_chemical(chem_text):
    chem_text = str(chem_text).lower()
    for keyword, label in chemical_mapping.items():
        if keyword in chem_text:
            return label
    return 'N/A'


cosmetics_main['chemicals'] = cosmetics_main['chemical'].apply(map_chemical)


In [26]:
cosmetics_main['brand'].unique().tolist()

['avon',
 "glover's",
 'opi',
 'absolute',
 'absolute fx',
 'gojo�',
 'chanel',
 "aloe up lil' kids spf 45 aloe based sunscreen lotion",
 'entity one uv gel',
 'entity nail enamel',
 'entity nail sculpting powder',
 'entity nail top coat',
 'almay',
 "aloe up lil' kids spf 30+ sunscreen stick",
 'aloe up spf 30 sunscreen stick',
 'dermalogica',
 'clarins paris',
 'revlon',
 'light elegance',
 'philosophy',
 'pca skin',
 'ysl',
 'maybelline',
 'anew',
 'mark',
 'elizabeth arden',
 'kandesn',
 'oi-lin',
 '2 cruel',
 'st. john',
 'micro-colors',
 'aqua',
 'elegance pigments',
 'image pigments',
 'double max',
 'triple max',
 'velvet',
 'double concentrate',
 'triple concentrate',
 'dermalogica - clean start',
 'dermalogica - chromawhite',
 'tahitian noni� tahiti trim� plan 40�',
 'nsi',
 'the body shop',
 'zest',
 'olay',
 'buxom',
 'bareminerals',
 'designline',
 'bliss',
 'merle norman cosmetics',
 'zotos intl, inc.',
 'barevitamins',
 'rareminerals',
 'bareescentuals',
 'md formulation

In [27]:
cosmetics_main.head(1)

,product,company,brand,category,subcategory,chemical,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,chemicals
0,ultra color rich extra plump lipstick-all shades,new avon llc,avon,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide


In [28]:
cosmetics_main2=cosmetics_main.drop(columns = ['chemical'])

In [29]:
cosmetics_main2['company'].to_list()

['new avon llc',
 'j. strickland & co.',
 'j. strickland & co.',
 'new avon llc',
 'new avon llc',
 'new avon llc',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'opi products inc.',
 'new avon llc',
 'gojo industries, inc.',
 'chanel, inc',
 'chanel, inc',
 'chanel, inc',
 'chanel, inc',
 'aloecare international, llc',
 'aloecare international, llc',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, inc.',
 'entity beauty, 

In [30]:
cosmetics_main2=cosmetics_main2.rename(columns={'brand_cleaned':'brand'})

In [31]:
cosmetics_main2

,product,company,brand,category,subcategory,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,chemicals
0,ultra color rich extra plump lipstick-all shades,new avon llc,avon,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide
1,glover's medicated shampoo,j. strickland & co.,glover's,Hair Care Products (non-coloring),Hair Shampoos (making a cosmetic claim),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2,False,coal
2,glover's medicated shampoo,j. strickland & co.,glover's,Hair Care Products (non-coloring),Hair Shampoos (making a cosmetic claim),07/01/2009,07/01/2009,NaN,07/02/2009,07/02/2009,NaN,2,False,estragole
3,precision glimmer eye liner-all shades �,new avon llc,avon,Makeup Products (non-permanent),Eyeliner/Eyebrow Pencils,07/09/2009,08/28/2013,NaN,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide
4,avon brilliant shine lip gloss-all shades �,new avon llc,avon,Makeup Products (non-permanent),Lip Gloss/Shine,07/09/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114630,hydra-lip translucent color lipstick,"yanbal usa, inc",yanbal,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,titanium dioxide
114631,hydra-lip translucent color lipstick,"yanbal usa, inc",yanbal,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,titanium dioxide
114632,hydra-lip translucent color lipstick,"yanbal usa, inc",yanbal,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,titanium dioxide
114633,hydra-lip translucent color lipstick,"yanbal usa, inc",yanbal,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,titanium dioxide


In [32]:
cosmetics_main2['company'].unique()

array(['new avon llc', 'j. strickland & co.', 'opi products inc.',
       'gojo industries, inc.', 'chanel, inc',
       'aloecare international, llc', 'entity beauty, inc.',
       'revlon consumer product corporation', 'dermalogica',
       'clarins s.a.', 'mcconnell labs, inc.', 'philosophy',
       "physician's care alliance, llc", "l'oreal usa",
       'elizabeth arden, inc.', 'sunrider manufacturing, l.p.',
       'romane fragrances', 'li pigments',
       'tahitian noni international, inc.', 'amco international',
       'buth-na-bodhaige, inc', 'the procter & gamble company',
       'bare escentuals beauty, inc.', 'regis corporation',
       'bliss world llc', 'merle norman cosmetics',
       'zotos international, inc.', 'arcadia beauty labs llc',
       'tigi linea corp', 'colomer u.s.a., inc.',
       'alfalfa nail supply, inc.', 'no lift nails inc.',
       'neostrata company, inc.', 'beauticontrol, inc.',
       'klein-becker usa, llc.', 'astara skin care', 'voss laboratorie

In [33]:
cosmetics_main2['company'].to_list()

def normalize(text):
    text = str(text).lower().strip()
    
    text = re.sub(r'[^a-z0-9]', '', text)
    return text

In [34]:
cosmetics_main2['company_normalized'] = cosmetics_main2['company'].apply(normalize)

In [35]:
company_mapping={
        'new avon llc': 'new avon llc', 'j. strickland & co.': 'j. strickland & co.', 'opi products inc.':'opi',
       'gojo industries, inc.':'gojo industries', 'chanel, inc': 'chanel',
       'aloecare international, llc':'aloecare international', 'entity beauty, inc.':'entity',
       'revlon consumer product corporation':'revlon', 'dermalogica':'dermalogica',
       'clarins s.a.':'clarins', 'mcconnell labs, inc.':'mcconnell', 'philosophy':'philosophy',
       "physician's care alliance, llc":'physician', "l'oreal usa" : "l'oreal",
       'elizabeth arden, inc.':'elizabeth arden', 'sunrider manufacturing, l.p.':'sunrider',
       'romane fragrances':'romane', 'li pigments':'li pigments',
       'tahitian noni international, inc.':'tahitian noni', 'amco international':'amco',
       'buth-na-bodhaige, inc':'buth-na-bodhaige', 'the procter & gamble company':'proctor & gamble',
       'bare escentuals beauty, inc.':'bare essentials', 'regis corporation':'regis',
       'bliss world llc':'bliss world', 'merle norman cosmetics':'merle norman',
       'zotos international, inc.':'zotos', 'arcadia beauty labs llc':'arcadia',
       'tigi linea corp':'tigi linea', 'colomer u.s.a., inc.':'colomer',
       'alfalfa nail supply, inc.':'alfalfa', 'no lift nails inc.':'no lift nails',
       'neostrata company, inc.':'neostrata', 'beauticontrol, inc.':'beauticontrol',
       'klein-becker usa, llc.':'klein-becker', 'astara skin care':'astara', 'voss laboratories':'voss',
       'c.f.e.b. sisley':'sisley', 'whiter image':'whiter image', 'bremenn research labs':'bremenn research labs',
       'noevir co., ltd':'noevir', 'young nails, inc.':'young nails', 'beiersdorf, inc.':'beiersdorf',
       'added extras llc':'added extras', 'giovanni cosmetics, inc.':'giovanni cosmetics',
       'beaute prestige international':'beaute prestige international', 'cool cuts 4 kids':'cool cuts 4 kids',
       'farouk systems, inc.':'farouk systems', 'br sas': 'br sas', 'signatry, inc.':'signatary',
       'chattem, inc.':'chattem', 'excelon':'excelon', 'georgia-pacific consumer products lp':'georgia-pacific',
       'coty':'coty', 'combe incorporated':'combe', 'nyx los angeles, inc.':'nyx',
       'primorance':'primorance', 'mazunte llc':'mazunte', 'palladio beauty group':'palladio',
       'namaste laboratories':'namaste', 's.c. johnson & son, inc.':'sc johnson',
       'ibs beauty inc':'ibs beauty', 'levlad llc':'levlad', 'hillyard industries':'hillyard',
       'aloette cosmetics inc.':'aloette cosmetics', 'benefit cosmetics':'benefit cosmetics',
       'kiss products, inc.':'kiss', 'tressa inc.':'tressa',
       'johnson & johnson consumer companies':'johnson & johnson',
       'origins natural resources inc.':'origins natural resources', 'thebalm':'thebalm',
       'alberto culver usa, inc.':'alberto culver', 'clinique laboratories, llc':'clinique',
       'estee lauder inc.':'estee lauder', 'melaleuca, inc.':'malaleuca', 'pure & basic products':'pure & basic products',
       'afflink':'afflink', 'john paul mitchell systems':'john paul mitchell', 'edward don & company':'edward don & company',
       'lagassesweet':'lagassesweet', 'henkel corporation':'henkel',
       'kimberly-clark global sales, llc':'kimberly-clark', 'unite eurotherapy':'unite eurothreapy',
       'tarte cosmetics':'tarte', 'conopco, inc.':'conopco', 'mannatech incorporated':'mannatech',
       'purminerals':'purminerals', 'cosmedix':'cosmedix', 'multaler & cie s.a.':'multaler et cie',
       'tupperware u.s., inc.':'tupperware', 'prescriptives inc.':'prescriptives',
       'san joaquin supply':'san joaquin supply', 'beautybank inc.':'beautybank',
       'sun pharmaceuticals, llc':'sun pharmaceuticals', 'darphin llc':'darphin',
       'physicians formula holdings, inc.':'physicians formula', 'kao usa inc.':'kao',
       'athena cosmetics, inc.':'athena cosmetics', 'supre':'supre', 'key brands international ltd.':'key brands international',
       'pbigroup inc':'pbigroup',
       'markwins international, markwins beauty products, physicians formula':'physicians formula',
       "laboratoires m&l sa (l'occitane - melvita)":"l'occitane",
       'colgate-palmolive company':'colgate-palmolive', 'molton brown ltd':'molton brown',
       'pss world medical, inc':'pss world medical', 'pacific world cosmetics':'pacific world cosmetics',
       'perron rigot':'perron rigot', 'kevin murphy':'kevin murphy', 'earth essentials':'earth essentials', 'aramis inc.':'aramis',
       'dhc usa inc.':'dhc', 'sexy hair':'sexy hair', 'jurlique international pty. ltd.':'jurlique',
       'laboratoire nuxe':'nuxe', 'davines s.p.a.':'davines', 'kao germany gmbh':'kao',
       'parfums givenchy s.a.':'givenchy', 'kanebo cosmetics inc':'kanebo',
       'shiseido america, inc.':'shiseido', 'harmon stores inc.':'harmon',
       'access business group international, llc':'shiseido', 'jo malone inc.':'jo malone',
       'ojon corporationa':'ojon', 'triple s':'triple s', 'corporate express':'corporate express',
       'diversey, inc':'diversey', 'beautopia llc':'beautopia', 'hallmark cards, inc.':'hallmark',
       'murad skin research laboratories, inc.':'murad', 'sonya dakar skin care':'sonya dakar',
       'skinmedica, an allergan company':'skinmedica', 'armand dupree inc.':'armand dupree',
       'orlane, inc':'orale', 'guerlain':'guerlain', 'marianna industries':'marianna',
       'le groupe fruits & passion inc.':'le groupe fruits & passion', 's+':'s+', 'mary kay inc.':'mary kay',
       'expanscience laboratoires':'expanscience', 'deb usa, inc.':'deb', 'great clips, inc.':'great clips',
       'puretek corporation':'puretek', "burt's bees inc.":"burt's bees", 'ivy enterprises, inc.':'ivy',
       'src':'src', 'kmc exim corporation/dashing diva franchise corp':'kmc',
       'i.c.o.n. line, inc.': 'i.c.o.n. line', 'general motors':'general motors',
       'nutrimetics australia pty limited':'nutrimetics australia', 'yves rocher inc.':'yves rocher',
       'bonne bell, llc':'bonne bell', 'nail magic llc':'nail magic', 'parfums christian dior':'chritian dior',
       'nse products, inc.':'nse products', 'thierry mugler parfums':'thierry mugler parfums', 'stila styles llc':'stila styles',
       'neways':'neways', 'bioelements, inc.':'bioelements', 'anastasia beverly hills, llc':'anastasia beverly hills',
       'energizer personal care, llc':'energizer personal care', 'coria laboritories':'coria',
       'california north':'california north', "penhaligon's limited":"penhaligon's limited", 'make up for ever':'makeup forever',
       'obagi medical products, inc.':'obagi medical products', 'xtreme color, inc.': 'xtreme color',
       'anc spa inc':'anc spa', 'carita international':'carita', 'laboratoires decleor': 'laboratoires decleor',
       'les salons du palais royal shiseido':'les salons du palais royal shiseido', 'crabtree & evelyn, ltd':'crabtree & evelyn',
       'fashion fair, llc':'fashion fair', 'stendhal':'stendhal', 'zo skin health, inc':'zo skin health',
       'kingdom animalia, llc.':'kingdom animalia', 'australian gold':'australian gold',
       'beauty society, inc.':'beauty society', 'korres s.a - natural products':'korres',
       'shiseido co., ltd.':'shiseido', 'bath & body works': 'bath & body works',
       'jafra cosmetics international':'jafra cosmetics', 'nars cosmetics':'nars',
       'tammy taylor nails, inc.':'tammy taylor nails', 'american international industries':'american international industries',
       'arbonne international llc': 'arbonne international', 'lvmh fragrance brands':'lvmh',
       'beauty selectives':'beauty selectives', 'gurwitch products':'gurwitch products', 'kings and queens ltd':'kings and queens',
       'proclean inc.':'proclean', 'the home depot':'home depot', 'us cosmeceutechs, llc': 'us cosmecutechs',
       'lvmh fragrance brands - kenzo parfums':'lvmh',
       'max huber research labs, inc.':'huber', 'pro-link inc':'pro-link',
       'prime source llc':'prime', 'w. w. grainger, inc.,':'w.w. grainer',
       'waxie sanitary supply':'waxie sanitary suppy', 'gap inc.':'gap', 'orly international':'orly',
       'cardinal health':'cardinal health', 'thymes llc':'thymes', 'caleel+hayden':'caleel+hayden',
       'herbalife international of america, inc.':'heerbalife',
       'bobbi brown professional cosmetics, inc.':'bobbi brown', 'makeup art cosmetics':'makeup art',
       'watkins, inc':'watkins', 'aveda corporation':'aveda', 'the kama sutra':'the karma sutra',
       'schwarzkopf, inc.':'schwarzkopf', 'erbaviva llc':'erebaviva', "victoria's secret beauty":"victoria's secret",
       'fisk industries inc':'fisk industries', 'the hain celestial group, inc':'the hain celestial group',
       'hayashi tecnica, llc':'hayashi tecnica', 'central solutions, inc.':'central solutions',
       'jemma kidd make up ltd':'jemma kidd make up', 'kmc exim':'kmc', 'mdsolarsciences �':'mdsolarsciences',
       'iredale mineral cosmetics':'iredale mineral cosmetics', 'cos.brands, llc':'cos.brands',
       'freeze 24/7 international, llc':'freeze 24/7 international', 'reckitt benckiser':'reckitt benckiser',
       'private label select ltd':'private label select', 'zalan products, inc.':'zalan products',
       'dreambrands, inc':'dreambrands', 'beilis development':'beilis development',
       'trivani international, l.l.c.':'trivani', 'tropical seas, inc.':'tropical seas',
       'framesi-north america':'framesi', 'ligi import corp.':'ligi import',
       'kingman industries, inc':'kingman industries', 'benev company, inc.':'benev',
       'babor cosmetics america corp':'babor',
       'university medical pharmaceuticals corp':'university medical pharmaceuticals', 'revision, llc':'revision',
       'ag professional hair care products ltd':'ag professional',
       'l&s cosmetics & toiletries (m) sdn bhd':'l&s cosmetics', 'hoyu america co.':'hoyu',
       'olympic mountain products':'olympic mountain products', 'cali chem inc':'cali chem',
       'eng kah enterprise sdn bhd':'eng kah enterprise', 'ccs, clean chemical sweden ab':'ccs',
       'e.t. browne drug co. inc.':'e.t. browne drug co.', 'dr. babor gmbh & co. kg':'dr.babor gmbh & co. kg',
       'germaine de capuccini':'germaine de capuccini', 'nv perricone llc':'nv perricone',
       'goodier cosmetics inc':'goodier', 'apollo health and beauty care':'apollo',
       'natura bisse international':'natura bisse', 'sun & skin care research, inc':'sun & skin care research',
       'cosswiss ag':'cosswiss ag', 'garcoa laboratories, inc.':'garcoa', 'archipelago inc.':'archipelago',
       'delon laboratories (1990) inc':'delon', 'pinnacle cosmetics':'pinnacle',
       'dermaquest, inc.':'dermaquest', "nature's sunshine products, inc.": "nature's sunshine products",
       'morris profumi spa': 'morris profumi spa', 'dr. nona international ltd':'dr.nona',
       'new fragrance continental inc.':'new fragrance continental', 'allure labs, inc.':'allure', 'walmart':'walmart',
       'california baby':'california', 'mirage cosmetics inc':'mirage cosmetics', 'antonio puig s.a.':'antonio puig',
       'new directions aromatics inc.':'new directions aromatics', 'cover fx skin care inc.':'cover fx skin care',
       'laboratoire sintyl sa':'laboratoire sintyl sa', 'beauty 21 cosmetics, inc.':'beauty 21 cosmetics',
       'custom amenities inc.':'custom amenities', 'smith & vandiver corporation':'smith & vandiver',
       'biocosm�tica exel argentina s.r.l.':'biocosmetic exel argentina', "tom's of maine, inc.":"tom's of maine",
       'scruples professional salon products, inc.':'scruples professional salon products',
       'ceras especiales martinez de san vicente sa':'ceras especiales martinez de san vicente sa',
       'mario zunino & c. s.r.l.':'mario zunino & c. s.r.l.', 'zohar cosmetics (m.f.o.) ltd.':'zohar cosmetics (m.f.o.)',
       'jean philippe fragrances inc':'jean philippe', 'smart brands inc':'smart brands',
       'rowpar pharmaceuticals, inc':'rowpar pharmaceuticals', 'the boots company plc':'the boots company',
       'italian group srl':'italian group', 'pharmaceutical specialties, inc.':'pharmaceutical specialties',
       'kenra professional, llc':'kenra professional', 'interparfums inc.':'interparfums', 'trade west, inc.':'trade west',
       'prostrong inc.':'prostrong', 'bio-derm laboratories, inc.':'bio-derm',
       'lise watier cosm�tiques inc.':'lise watier cosm�tiques', 'true cosmetics, llc':'true cosmetics',
       'b�rlind of germany, inc.':'b�rlind of germany', 'sysco guest supply llc.':'sysco guest supply',
       'lca - laboratori di chimica applicata srl':'lca',
       'boom creative development llc':'boom creative development', 'advanced aesthetics, inc.':'advanced aesthetics',
       'robell research':'robell research', 'aromca s.a':'aromca', 'smashbox cosmetics':'smashbox',
       'world club suppply corp.':'world club supply', 'ipd inc':'ipd', "zach's wax":"zax's wax",
       'fragrances of ireland ltd.':'fragrances of ireland', 'neoteric cosmetics inc':'neoteric',
       "lauren hutton's good stuff llc":"laurn hutton's good stuff",
       'beauty treats international co., inc.':'beauty treats international',
       'the village company, llc':'the village company', 'vi-jon, inc.':'vi-jon',
       'selected chemical products company':'selected chemical products', 'skinn cosmetics, llc':'skinn cosmetics',
       'fresh, inc.':'fresh', 'continental fragrances, ltd.':'continental',
       'ulta salon, cosmetics & fragrance, inc.':'ulta', 'lumene oy':'luemene',
       'pettenon cosmetici srl':'pettenon', 'parfums de coeur, ltd.':'parfums de coeur',
       'studio gear cosmetics, inc.':'studio gear',
       'private brand, consumer products, division of mckesson':'mckesson',
       'concept laboratories, inc.':'concept laboratories', 'aubrey organics inc':'aubrey organics',
       'avlon industries, inc.':'avlon', 'exclusive fragrances & cosmetics':'exclusive fragrances & cosmetics',
       'paco rabanne parfums':'paco rabanne parfums', 'parfums nina ricci':'parfums nina ricci', 'scherer, inc.':'scherer',
       'go smile, inc.':'go smile', 'american consumer products, llc':'american consumer products',
       'mor cosmetics international llc':'mor cosmetics international', 'b & c products, inc.':'b & c products',
       'laboratoires dr n.g. payot':'laboratoires dr n.g. payot', 'elysee cosmetics':'elysee',
       'lush manufacturing ltd.':'lush', 'fragrance and skincare':'fragrance and skincare',
       'laboratoire biosthetique kosmetik gmbh & co.':'laboratoire biosthetique kosmetik gmbh & co.', 'rolland srl':'rolland',
       'country life, llc':'country life', 'unisource worldwide, inc':'unisource',
       'science of skincare, llc':'science of skincare', 'mckesson medical-surgical':'mckesson',
       'cypress medical products':'cypress', 'zee medical, inc':'zee',
       'ventura international, ltd':'ventura', 'cca industries, inc.':'cca',
       'the sanctuary spa holdings ltd':'the sanctuary sap', 'the tend skin company':'tend skin',
       'the kroger co.':'kroger', 'new spirit naturals, inc.':'new spirit naturals', 'blistex inc.':'blistex',
       "good 'n natural":'good & natural', 'herbal harvest, inc.':'herbal harvest', 'herbal authority':'herbal authority',
       'home health':'home health', 'natural wealth':'natural wealth', "nature's bounty":"nature's bounty",
       'vitamin world, inc.':'vitamin world', 'physiologics':'physiologics', "puritan's pride":"puritan's pride",
       'radiance':'radiance', 'rexall-sundown':'rexall-sundown', 'chantecaille beaut� inc':'chantecaille beaut�',
       'greenbrier international, inc.':'greenbrier', 'swanson health products':'swanson',
       'salerm cosm�tica profesional,inc.':'salerm cosm�tica profesional', 'sothys usa inc':'sothys',
       'rapha racing ltd.':'ralpha racing', 'pbh marketing inc.':'pbh',
       'kaoder industry company, ltd':'kaoder industry', 'green mountain biotech co., ltd':'green mountain biotech',
       'woosin cosmetics co., ltd':'woosin',
       'valeant pharmaceuticals north america':'valeant', 'lt white, inc.':'lt white',
       'wellmade household & health care co., ltd':'wellmade household & health care',
       'trish mcevoy cosmetics limited':'trish mcevoy', 'ziba beauty':'ziba',
       'design plus health & beauty ltd':'design plus health & beauty',
       'jan marini skin research, inc.':'jan marini skin', 'msd consumer care, inc.':'msd',
       'montagne jeunesse':'montagne jeunesse', 'a. p. deauville, llc':'a.p. deauville', 'izar sarl':'izar sarl',
       'parfums loris azzaro':'loris azzaro', 'fusion brands inc.':'fushion brands',
       'natural solter s.l.':'natural solter', 'i love cosmetics limited':'i love cosmetics limited', 'hada s.a.':'hada',
       'alberto culver':'alberto culver', 'soap and glory ltd': 'soap & glory',
       'kate somerville skin care':'kate somerville', 'femme couture international, inc.':'femme couture',
       'atlas development limited':'atlas', 'kmart corporation':'kmart', 'supervalu inc.':'supervalu',
       'topco associates, llc':'topco', 'rite aid corporation':'rite aid',
       'alessandro international gmbh':'alessandro', 'cameo trading co., inc.':'cameo trading',
       'neocutis, inc':'neocutis', 'gib, llc':'gib', 'enchante accessories, inc.':'enchante accessories',
       'champneys henlow limited':'champneys henlow', 'athena cosmetics':'athena', 'prima-derm, s.l.':'prima-derm',
       'beauty solutions, ltd.':'beauty solutions', 'rexall, inc.':'rexall', 'perfectly pure':'perfectly pure',
       'gdcp international ltd':'gdcp', 'betco corporation, ltd':'betco', 'fresh inc.':'fresh',
       'american consumer products':'american consumer', 'look beauty products':'look beauty',
       'yiwu aris cosmetic manufactory':'yiwu aris cosmetic',
       'nippon menard cosmetic co., ltd.':'nippon menard', 'colorescience':'colorescience', 'core brands':'core brands',
       'taizhou xinzhixuan daily-use co.,ltd':'taizhou xinzhixuan daily-use',
       'hangzhou beja commodity co., ltd':'hangzhou beja commodity', 'the browgal':'the browgal',
       'dollar connection ltd':'dollar connection', 'california mango':'california mango',
       'kendo holdings, inc.':'kendo holdings', 'prai beauty group inc':'prai beauty group',
       'demeter fragrance library, inc.':'demeter fragrance library', 'r and t creation ltd':'r and t creation',
       'rt group':'rt group', 'test':'test', 'rituals cosmetics usa, inc.':'rituals cosmetics',
       'zhejiang yiwu lucy cosmetics co.,ltd.':'zhejiang yiwu lucy cosmetics',
       'ninghai jixiang stationery factory':'ninghai jixiang stationery factory',
       'dongguan changping mei ya cosmetics factory':'dongguan changping mei ya cosmetics', 'deborah lippmann':'ddeborah lippmann',
       'calibre beaut�, a division of quadrant cosmetics corp.':'calibre beaut�',
       'laboratoires dr n.g payot':'laboratoires dr n.g payot', 'novelty, inc.':'novelty',
       'too faced cosmetics':'too faced', 'yantai dupont daily necessities co., ltd.':'yantai dupont daily necessities co.',
       'japonesque':'japonesque', 'amway corporation':'amway', "paula's choice llc.":"paula's choice",
       'guthy-renker, llc':'guthy renker', 'modere, inc.':'modere', 'space brands limited':'space brands limited',
       'the mentholatum company':'the mentholatum', 'interparfums':'interparfums', 'high ridge brands':'high ridge brands',
       'express, llc':'express', 'lush ltd':'lush', 'urban decay cosmetics':'urban decay', 'alqvimia':'alqvimia',
       'aesop usa inc.':'aesop', 'rodan + fields':'rodan + fields', 'hatchbeauty products, llc':'hatchbeauty',
       'frank body':'frank body', 'shiseido americas corporation':'shiseido',
       'kryolan corporation':'kryolan', 'upper canada soap & candle makers corp.':'upper canada soap & candle makers',
       'label.m usa inc':'label.m', 'guangzhou zixuan beauty equipment co.,ltd':'guangzhou zixuan beauty',
       'hand & nail harmony, inc':'hand & nail harmony', 'red carpet manicure':'red carpet manicure',
       'artistic nail design - nail alliance llc':'nail alliance', 'nail alliance, llc':'nail alliance',
       'nail alliance - entity':'nail alliance',
       'neostrata company, inc., a johnson & johnson company':'neostrata',
       'maesa llc':'maesa', 'perfect angel cosmetics & health co., limited':'perfect angel cosmetics & health co.',
       'yes to inc.':'yes to', 'walt disney parks & resorts':'walt disney parks & resorts', 'nudestix':'nudestix',
       'the himalaya drug company':'the himalaya drug company', 'sexy hair concepts':'sexy hair concepts', 'rms beauty':'rms beauty',
       'luxury brand partners':'luxury brand partners', 'tweezerman international llc':'tweezerman',
       'aplicare, inc':'aplicare', 'black up':'black up', 'eos products, llc':'eos', 'soma':'soma',
       'alphaeon corporation':'alphaeon', 'amorepacific us inc.':'amorepacific', 'm.t. mocara gmbh':'m.t. mocara',
       'la prairie, inc.':'la praire', 'kos� america, inc.':'kos', 'brodie & stone':'brodie & stone',
       'labo international srl':'labo', 'arcadia beauty labs, llc':'arcadia',
       'hollywood alliance':'hollywood alliance', 'milk make-up':'milk make-up', 'we are luxe ltd.':'we are luxe',
       'malin & goetz':'malin & goetz', 'reforma group':'reforma group', 'felps usa, inc':'felps',
       'saje natural business inc.':'saje natural', 'sd biotechnologies usa, inc.':'biotechnologies',
       'imperial dax hair care inc.':'imperial dax hair care', 'person & covey':'person & covey',
       'fantasia industries':'fantasia', 'house of cheatham':'house of cheatham', 'michel design works':'michel design works',
       "trader joe's company":"trader joe's company", 'drunk elephant llc':'drunk elephant', 'chrome hearts llc':'chrome hearts',
       'cosmetic dermatology inc':'cosmetic dermatology', 'creative nail design, inc.':'creative nail design',
       'herbswork cc':'herbswork', 'townley inc':'townley', 'centric beauty, llc':'centric', 'ppp&c inc.':'ppp&c',
       'charlotte tilbury beauty ltd':'charlotte tilbury', 'mavala s.a.':'mavala', 'serysi usa inc':'serysi',
       'milani cosmetics':'milani', 'dr. hauschka skin care, inc.':'dr.hauschka skin care', 'zelens':'zelens',
       'memebox corporation':'memebox', 'vi-jon, inc':'vi-jon',
       'apollo health and beauty care inc.':'apollo', 'spa de soleil':'spa de soleil',
       'biotouch, inc.':'biotouch', 'trademark cosmetics, inc.':'tradeark cosmetics',
       'ca-botana international':'ca-botana', 'dlc laboratories, inc.':'dlc labortories', 'fmk labs':'fmk labs',
       'bulgari global operations sa':'bulgari', 'cover fx skincare inc':'cover fx skincare',
       'pearlosophy usa, llc':'pearlosophy', 'borboleta beauty':'borboleta', 'wahl clipper corp.':'wahl clipper',
       'huda beauty':'huda', 'the honest company':'the honest company', 'tatcha llc':'tatcha', 'plcosmetic':'plcosmetic',
       'osea malibu':'osea malibu', 'fresh inc':'fresh', 'acqua di parma':'acqua di parma', 'hello body gmbh':'hello body',
       'zoeva gmbh':'zovea', 'lg household & health care america inc.':'lg household & health care',
       'best accessory group':'best accessory', 'jinhua b&f cosmetics co., ltd.':'jinhua b&f cosmetics',
       '2355644 ontario inc.':'2355644 ontario inc', 'albion cosmetics (america), inc.':'albion cosmetics',
       'dba sas':'dba', 'live flickable, llc':'live flickable', 'perricone md':'perricone',
       'farmaceutici dott. ciccarelli spa':'farmaceutici dott. ciccarelli spa', 'the lano company':' the lano company',
       'mirabella':'mirabella', 'mtg co., ltd.':'mtg', 'ningbo meiteli cosmetics co.,ltd.':'ningbo meiteli cosmetics',
       'dollar shave club':'dollar shave club', 'esalon.com llc':'esalon.com', 'rea.deeming beauty, inc.':'rea.deeming beauty',
       'yanbal usa, inc':'yanabal usa', 'ladove inc':'ladove', 'makingcosmetics inc.':'makingcosmetics',
       'carma laboratories':'carma', 'hayashi worldwide':'hayashi', 'bielle cosmetics, inc':'bielle',
       'whisk products, inc':'whisk', 'aloe vera of america, inc.':'aloe vera of america',
       'edgewell personal care brands, llc':'edgewell', 'sunflower distributing':'sunflower distributing',
       'mana products inc.':'mana products', 'your name professional brands':'your name professional brands',
       'taste beauty llc':'taste', 'shikai products':'shikai', 'instanatural llc':'instanatural',
       'pharmco laboratories':'pharmco', 'usana health sciences, inc.':'usana health',
       'momotani juntenkan ltd.':'momotani juntenkan', 'inglot usa llc':'inglot', 'duri cosmetics inc':'duri',
       'ningbo rejoice i/e co., ltd.':'ningbo rejoice', 'vb cosmetics, inc.':'vb',
       'ultimate face cosmetics, inc':'ultimate face', 'robanda international':'robanda',
       'lornamead':'lornamead', 'pfizer consumer healthcare':'pfizer', 'market ready':'market ready',
       'colbert+':'colbert+', 'ultraceuticals pty ltd':'ultaceuticals',
       'rainbow cosmetics manchester ltd':'rainbow cosmetics manchester', 'g.o holdings, inc.':'g.o holdings',
       'petlab cosmetics, llc':'petlab', "the nature's bounty company":"nature's bounty",
       'h2o plus, llc':'h20 plus', 'pacha soap co.':'pacha', 'oceana usa, inc.':'oceana',
       'beach house group':'beach house group', 'oribe hair care llc':'oribe hair carfe', 'it girl beauty, llc':'it girl beauty',
       'h&m gutberlet gmbh':'h&m', 'stila style llc.':'stila',
       'sanmedica international, llc':'sanmedica', 'interparfums usa, llc':'interparfums',
       'e. excel, llc':'e.excel', 'strivectin operating company, inc.':'strivectin operating',
       'jiangxi iris chemical co.,ltd':'jiangxi iris chemical co.', 'dnd daisy nail products, inc.':'dnd',
       'zschimmer & schwarz, inc.' : 'zschimmer & schwarz', 'cosmopharm ltd.' : 'cosmo pharmaceuticals',
       'guangzhou baiyun darong fine chemical industry co.,ltd.':'guangzhou baiyun darong',
       'turquise global llc': 'turquoise global'

}

In [36]:
cosmetics_main2['company_standardized'] = cosmetics_main2['company'].str.lower().map(company_mapping).fillna(cosmetics_main2['company'])

In [37]:
cosmetics7=cosmetics_main2

In [38]:
cosmetics_main2.head()

,product,company,brand,category,subcategory,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,chemicals,company_normalized,company_standardized
0,ultra color rich extra plump lipstick-all shades,new avon llc,avon,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide,newavonllc,new avon llc
1,glover's medicated shampoo,j. strickland & co.,glover's,Hair Care Products (non-coloring),Hair Shampoos (making a cosmetic claim),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2,False,coal,jstricklandco,j. strickland & co.
2,glover's medicated shampoo,j. strickland & co.,glover's,Hair Care Products (non-coloring),Hair Shampoos (making a cosmetic claim),07/01/2009,07/01/2009,NaN,07/02/2009,07/02/2009,NaN,2,False,estragole,jstricklandco,j. strickland & co.
3,precision glimmer eye liner-all shades �,new avon llc,avon,Makeup Products (non-permanent),Eyeliner/Eyebrow Pencils,07/09/2009,08/28/2013,NaN,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide,newavonllc,new avon llc
4,avon brilliant shine lip gloss-all shades �,new avon llc,avon,Makeup Products (non-permanent),Lip Gloss/Shine,07/09/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide,newavonllc,new avon llc


In [39]:
cosmetics7.to_csv('cosmetics7.csv')

In [40]:
def normalize_brand(brand):
    brand = str(brand).lower()
    brand = re.sub(r'[^a-z0-9\s]', '', brand)
    brand = re.sub(r'\s+', ' ', brand).strip()
    return brand

In [41]:
cosmetics_main2['normalized_brand'] = cosmetics_main2['brand'].apply(normalize_brand)

In [42]:
def simple_cluster(brands, threshold=0.9):
    mapping = {}
    for brand in brands:
        if brand in mapping:
            continue
        matches = get_close_matches(brand, brands, n=len(brands), cutoff=threshold)
        for match in matches:
            mapping[match] = brand
    return mapping


In [43]:
unique_brands = cosmetics_main2['normalized_brand'].dropna().unique().tolist()
brand_mapping = simple_cluster(unique_brands)

In [44]:
cosmetics_main2['standardized_brand'] = cosmetics_main2['normalized_brand'].map(brand_mapping)

In [45]:
the_main_cosmetics=cosmetics_main2

In [46]:
the_main_cosmetics=the_main_cosmetics.drop(columns=['company','brand'])

In [47]:
the_main_cosmetics.columns

Index(['product', 'category', 'subcategory', 'initial_reported_date',
       'most_recent_date_reported', 'discontinued_date',
       'chemical_created_date', 'chemical_updated_date', 'chemical_removed',
       'chemicalcount', 'titanium_dioxide', 'chemicals', 'company_normalized',
       'company_standardized', 'normalized_brand', 'standardized_brand'],
      dtype='object')

In [48]:
the_main_cosmetics=the_main_cosmetics.rename(columns={'company_standardized':'company','standardized_brand':'brand'})

In [49]:
the_main_cosmetics=the_main_cosmetics.drop(columns=['normalized_brand','company_normalized'])

In [50]:
the_main_cosmetics.head(2)

,product,category,subcategory,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,chemicals,company,brand
0,ultra color rich extra plump lipstick-all shades,Makeup Products (non-permanent),"Lip Color - Lipsticks, Liners, and Pencils",06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,titanium dioxide,new avon llc,avon
1,glover's medicated shampoo,Hair Care Products (non-coloring),Hair Shampoos (making a cosmetic claim),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2,False,coal,j. strickland & co.,glovers


In [51]:
the_main_cosmetics['subcategory'].value_counts().head(5)

subcategory
Lip Color - Lipsticks, Liners, and Pencils    16555
Eye Shadow                                    15744
Foundations and Bases                         14096
Nail Polish and Enamel                         9945
Lip Gloss/Shine                                8972
Name: count, dtype: int64

In [52]:
def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [53]:
the_main_cosmetics['normalized_subcategory'] = the_main_cosmetics['subcategory'].apply(normalize_text)

In [54]:
def simple_cluster(values, threshold=0.9):
    mapping = {}
    for val in values:
        if val in mapping:
            continue
        matches = get_close_matches(val, values, n=len(values), cutoff=threshold)
        for match in matches:
            mapping[match] = val
    return mapping

In [55]:
the_main_cosmetics['normalized_subcategory'].unique()

array(['lip color lipsticks liners and pencils',
       'hair shampoos making a cosmetic claim', 'eyelinereyebrow pencils',
       'lip glossshine', 'eye shadow',
       'artificial nails and related products', 'mascaraeyelash products',
       'hand cleansers and sanitizers',
       'sunscreen making a cosmetic claim', 'uv gel nail polish',
       'basecoats and undercoats', 'scrubs and exfoliants',
       'foundations and bases', 'nail polish and enamel',
       'antiwrinkleantiaging products making a cosmetic claim',
       'skin moisturizers making a cosmetic claim',
       'acne products making a cosmetic claim', 'facial masks',
       'lip balm making a cosmetic claim',
       'skin astringent making a cosmetic claim',
       'skin bleaching making a cosmetic claim',
       'perfumes solids and powders', 'blushes',
       'bubble and foam bath products', 'skin cleansers',
       'body washes and soaps', 'rouges', 'face powders',
       'skin toner making a cosmetic claim',
      

In [56]:
subcategory_mapping = {'lip color lipsticks liners and pencils':'lip',
       'hair shampoos making a cosmetic claim':'hair', 'eyelinereyebrow pencils':'eye',
       'lip glossshine':'lip', 'eye shadow':'eye',
       'artificial nails and related products':'nails', 'mascaraeyelash products':'eye',
       'hand cleansers and sanitizers':'sanitizer',
       'sunscreen making a cosmetic claim':'sun', 'uv gel nail polish':'nails',
       'basecoats and undercoats':'nails', 'scrubs and exfoliants':'body',
       'foundations and bases':'face', 'nail polish and enamel':'nails',
       'antiwrinkleantiaging products making a cosmetic claim':'face',
       'skin moisturizers making a cosmetic claim':'moisturizer',
       'acne products making a cosmetic claim':'face', 'facial masks':'face',
       'lip balm making a cosmetic claim':'lip',
       'skin astringent making a cosmetic claim':'body',
       'skin bleaching making a cosmetic claim':'body',
       'perfumes solids and powders':'body', 'blushes':'face',
       'bubble and foam bath products':'bath', 'skin cleansers':'body',
       'body washes and soaps':'bath', 'rouges':'other', 'face powders':'face',
       'skin toner making a cosmetic claim':'face',
       'tattoos and permanent makeup':'body',
       'powders excluding aftershave baby powder and makeup face powder':'body',
       'hair styling products':'hair', 'other nail products':'nails',
       'nighttime skin care':'body', 'indoor tanning products':'body',
       'other makeup product':'other', 'hair dyes and colors':'hair',
       'teeth cleaning products':'teeth', 'nail polish and enamel removers':'nails',
       'makeup fixatives':'face', 'perfumes oils and lotions':'fragrance',
       'hair conditioners rinseout':'hair', 'other skin care product':'body',
       'shaving cream and other beard softeners':'body', 'cuticle softeners':'nails',
       'nail creams and lotions':'nails', 'skin fresheners':'body', 'cologne':'fragrance', 'douches':'body',
       'bath additives':'bath', 'hair tints and rinses coloring':'hair',
       'mouthwashes and breath fresheners':'teeth', 'baby shampoos':'baby',
       'facial cream':'face', 'hair bleaches':'hair', 'hair conditioners leavein':'hair',
       'hair straighteners':'hair', 'other personal care product':'other',
       'permanent waves and wave sets':'hair', 'other hair care product':'hair',
       'foot powders and sprays':'body', 'depilatories':'body', 'paints eg facial body':'body',
       'hair lighteners with color':'hair', 'aftershave products':'body',
       'feminine deodorants':'body', 'sprays excluding fragrances':'fragrance',
       'baby washsoap':'baby', 'suntan enhancers':'sun', 'underarm deodorants':'fragrance',
       'perfumeseaux de parfum':'fragrance', 'baby skin care':'baby',
       'toilet watereaux de toilette':'fragrance', 'other bath products':'bath',
       'hair shampoos with antidandruff properties':'hair', 'other fragrances':'fragrance',
       'diaper rash treatment':'baby', 'teeth whitening products':'teeth',
       'other baby products':'baby', 'hair color sprays aerosol':'hair',
       'makeup preparations':'face', 'antiperspirants making a cosmetic claim':'fragrance',
       'other shaving product':'body', 'nail decoration':'nail',
       'products related to hair coloring':'hair',
       'lubricants eg personal sexual massage oil':'body',
       'other oral hygiene product':'teeth', 'other hair coloring product':'hair',
       'other sunrelated product':'sun'
                
    
                      }

In [57]:
the_main_cosmetics['standardized_subcategory'] = the_main_cosmetics['normalized_subcategory'].map(subcategory_mapping)

In [58]:
the_main_cosmetics['standardized_subcategory'] = the_main_cosmetics['normalized_subcategory'].map(subcategory_mapping)

In [59]:
the_main_cosmetics=the_main_cosmetics.drop(columns=['normalized_subcategory','subcategory'])

In [60]:
the_main_cosmetics=the_main_cosmetics.rename(columns={'standardized_subcategory':'subcategory'})

In [61]:
the_main_cosmetics.columns

Index(['product', 'category', 'initial_reported_date',
       'most_recent_date_reported', 'discontinued_date',
       'chemical_created_date', 'chemical_updated_date', 'chemical_removed',
       'chemicalcount', 'titanium_dioxide', 'chemicals', 'company', 'brand',
       'subcategory'],
      dtype='object')

In [62]:
from fuzzywuzzy import fuzz, process

def normalize(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

for col in ['brand', 'company', 'chemicals']:
    norm_col = f'normalized_{col}'
    the_main_cosmetics[norm_col] = the_main_cosmetics[col].apply(normalize)


def fuzzy_fix_column(df, norm_col, threshold=90):
    values = df[norm_col].dropna().unique()
    mapping = {}
    for val in values:
        if val in mapping:
            continue
        matches = process.extract(val, values, scorer=fuzz.token_sort_ratio)
        for match, score in matches:
            if score >= threshold:
                mapping[match] = val
    return mapping


brand_map = fuzzy_fix_column(the_main_cosmetics, 'normalized_brand')
company_map = fuzzy_fix_column(the_main_cosmetics, 'normalized_company')
chemical_map = fuzzy_fix_column(the_main_cosmetics, 'normalized_chemicals')


the_main_cosmetics['clean_brand'] = the_main_cosmetics['normalized_brand'].map(brand_map)
the_main_cosmetics['clean_company'] = the_main_cosmetics['normalized_company'].map(company_map)
the_main_cosmetics['clean_chemical'] = the_main_cosmetics['normalized_chemicals'].map(chemical_map)


the_main_cosmetics.drop(columns=['normalized_brand', 'normalized_company', 'normalized_chemicals'], inplace=True)


In [63]:
themaincosmetics=the_main_cosmetics.drop(columns=['brand','company','chemicals'])

In [64]:
themaincosmetics.head(1)

,product,category,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,subcategory,clean_brand,clean_company,clean_chemical
0,ultra color rich extra plump lipstick-all shades,Makeup Products (non-permanent),06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,lip,avon,new avon llc,titanium dioxide


In [65]:
themaincosmetics.columns

Index(['product', 'category', 'initial_reported_date',
       'most_recent_date_reported', 'discontinued_date',
       'chemical_created_date', 'chemical_updated_date', 'chemical_removed',
       'chemicalcount', 'titanium_dioxide', 'subcategory', 'clean_brand',
       'clean_company', 'clean_chemical'],
      dtype='object')

In [66]:
themaincosmetics=themaincosmetics.rename(columns={'clean_brand':'brand','clean_company':'company','clean_chemical':'chemical'})

In [67]:
themaincosmetics

,product,category,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,subcategory,brand,company,chemical
0,ultra color rich extra plump lipstick-all shades,Makeup Products (non-permanent),06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,lip,avon,new avon llc,titanium dioxide
1,glover's medicated shampoo,Hair Care Products (non-coloring),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2,False,hair,glovers,j strickland co,coal
2,glover's medicated shampoo,Hair Care Products (non-coloring),07/01/2009,07/01/2009,NaN,07/02/2009,07/02/2009,NaN,2,False,hair,glovers,j strickland co,estragole
3,precision glimmer eye liner-all shades �,Makeup Products (non-permanent),07/09/2009,08/28/2013,NaN,07/09/2009,07/09/2009,NaN,1,True,eye,avon,new avon llc,titanium dioxide
4,avon brilliant shine lip gloss-all shades �,Makeup Products (non-permanent),07/09/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1,True,lip,avon,new avon llc,titanium dioxide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114630,hydra-lip translucent color lipstick,Makeup Products (non-permanent),06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,lip,yanbal,yanabal usa,titanium dioxide
114631,hydra-lip translucent color lipstick,Makeup Products (non-permanent),06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,lip,yanbal,yanabal usa,titanium dioxide
114632,hydra-lip translucent color lipstick,Makeup Products (non-permanent),06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,lip,yanbal,yanabal usa,titanium dioxide
114633,hydra-lip translucent color lipstick,Makeup Products (non-permanent),06/19/2020,06/19/2020,NaN,06/19/2020,06/19/2020,NaN,1,True,lip,yanbal,yanabal usa,titanium dioxide


In [68]:
productstitanium=themaincosmetics.loc[themaincosmetics['titanium_dioxide']==True]

In [69]:
companytitanium=productstitanium['company'].value_counts().reset_index()

In [70]:
companytitanium=companytitanium.rename(columns= {'count':'titanium_count'})

In [71]:
cosmeticscounts=themaincosmetics['company'].value_counts().reset_index()
cosmeticscounts=cosmeticscounts.rename(columns={'count':'product_count'})

In [72]:
titaniumcompany=pd.merge(companytitanium,cosmeticscounts,on='company')

In [73]:
titaniumcompany=titaniumcompany[titaniumcompany['product_count']>titaniumcompany['titanium_count']]

In [74]:
titaniumcompany['percent_titanium_dioxide']= titaniumcompany['titanium_count']/titaniumcompany['product_count']
titaniumcompany['percent_titanium_dioxide']=titaniumcompany['percent_titanium_dioxide'].sort_values(ascending=False)

In [75]:
titaniumcompany['percent_titanium_dioxide']=titaniumcompany['percent_titanium_dioxide'].apply(lambda x: '{:.2%}'.format(x))

In [76]:
titaniumcompany=titaniumcompany.sort_values('percent_titanium_dioxide',ascending=False).reset_index()

In [77]:
top5=themaincosmetics['subcategory'].value_counts().reset_index().subcategory

In [78]:
top5=themaincosmetics.loc[themaincosmetics.subcategory.isin(top5)]

In [79]:
top5_df=top5.groupby('subcategory')['chemical'].value_counts().reset_index()

In [80]:
top5_df['rank']=top5_df.groupby('subcategory')['count'].rank(ascending=False)

**Start here**

In [82]:
themaincosmetics_discontinued=themaincosmetics

In [83]:
themaincosmetics_discontinued['discontinued_date'] = pd.to_datetime(themaincosmetics['discontinued_date'], errors='coerce')
themaincosmetics['discontinued_year'] = themaincosmetics['discontinued_date'].dt.year

In [84]:
themaincosmetics['discontinued_year']=themaincosmetics_discontinued['discontinued_year']

In [85]:
themaincosmetics_discontinued['initial_reported_date'] = pd.to_datetime(themaincosmetics['initial_reported_date'], errors='coerce')
themaincosmetics['initial_reported_year'] = themaincosmetics['initial_reported_date'].dt.year

In [86]:
themaincosmetics_discontinued['most_recent_date_reported'] = pd.to_datetime(themaincosmetics['most_recent_date_reported'], errors='coerce')
themaincosmetics['most_recent_date_reported_year'] = themaincosmetics['most_recent_date_reported'].dt.year

In [87]:
themaincosmetics_discontinued['chemical_created_date'] = pd.to_datetime(themaincosmetics['chemical_created_date'], errors='coerce')
themaincosmetics['chemical_created_year'] = themaincosmetics['chemical_created_date'].dt.year

In [88]:
themaincosmetics_discontinued['chemical_updated_date'] = pd.to_datetime(themaincosmetics['chemical_updated_date'], errors='coerce')
themaincosmetics['chemical_updated_year'] = themaincosmetics['chemical_updated_date'].dt.year

In [89]:
themaincosmetics_discontinued['chemical_removed'] = pd.to_datetime(themaincosmetics['chemical_removed'], errors='coerce')
themaincosmetics['chemical_removed_year'] = themaincosmetics['chemical_removed'].dt.year

In [90]:
themaincosmetics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114635 entries, 0 to 114634
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   product                         114635 non-null  object        
 1   category                        114635 non-null  object        
 2   initial_reported_date           114635 non-null  datetime64[ns]
 3   most_recent_date_reported       114635 non-null  datetime64[ns]
 4   discontinued_date               12920 non-null   datetime64[ns]
 5   chemical_created_date           114635 non-null  datetime64[ns]
 6   chemical_updated_date           114635 non-null  datetime64[ns]
 7   chemical_removed                2985 non-null    datetime64[ns]
 8   chemicalcount                   114635 non-null  int64         
 9   titanium_dioxide                114635 non-null  object        
 10  subcategory                     114635 non-null  object 

In [91]:
themaincosmetics['discontinued_year']=themaincosmetics['discontinued_year'].fillna(0).astype(int)

In [92]:
themaincosmetics['chemical_removed_year']=themaincosmetics['chemical_removed_year'].fillna(0).astype(int)

In [93]:
themaincosmetics.head(1)

,product,category,initial_reported_date,most_recent_date_reported,discontinued_date,chemical_created_date,chemical_updated_date,chemical_removed,chemicalcount,titanium_dioxide,subcategory,brand,company,chemical,discontinued_year,initial_reported_year,most_recent_date_reported_year,chemical_created_year,chemical_updated_year,chemical_removed_year
0,ultra color rich extra plump lipstick-all shades,Makeup Products (non-permanent),2009-06-17,2013-08-28,2011-02-01,2009-07-09,2009-07-09,NaT,1,True,lip,avon,new avon llc,titanium dioxide,2011,2009,2013,2009,2009,0


In [94]:
themaincosmetics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114635 entries, 0 to 114634
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   product                         114635 non-null  object        
 1   category                        114635 non-null  object        
 2   initial_reported_date           114635 non-null  datetime64[ns]
 3   most_recent_date_reported       114635 non-null  datetime64[ns]
 4   discontinued_date               12920 non-null   datetime64[ns]
 5   chemical_created_date           114635 non-null  datetime64[ns]
 6   chemical_updated_date           114635 non-null  datetime64[ns]
 7   chemical_removed                2985 non-null    datetime64[ns]
 8   chemicalcount                   114635 non-null  int64         
 9   titanium_dioxide                114635 non-null  object        
 10  subcategory                     114635 non-null  object 

In [95]:
carcinogens=pd.read_csv('list_of_chemicals.csv')

In [96]:
carcinogens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CAS No.                  904 non-null    object 
 1   Agent                    1120 non-null   object 
 2   Group                    1052 non-null   object 
 3   Volume                   1062 non-null   object 
 4   Volume publication year  1060 non-null   object 
 5   Evaluation year          1060 non-null   float64
 6   Additional information   93 non-null     object 
dtypes: float64(1), object(6)
memory usage: 61.4+ KB


In [97]:
for ind, row in carcinogens.iterrows():
    group_str = str(row['Group'])
    
    if '1' in group_str:
        carcinogens.loc[ind, 'Causes_cancer'] = 'Carcinogenic to humans'
    elif '2A' in group_str:
        carcinogens.loc[ind, 'Causes_cancer'] = 'Probably Carcinogenic to humans'
    elif '2B' in group_str:
        carcinogens.loc[ind, 'Causes_cancer'] = 'Possibly Carcinogenic to humans'
    elif '3' in group_str:
        carcinogens.loc[ind, 'Causes_cancer'] = 'Not classifiable as to its carcinogenicity to humans'
    else:
        carcinogens.loc[ind, 'Causes_cancer'] = 'N/A'

In [98]:
carcinogens.columns

Index(['CAS No.', 'Agent', 'Group', 'Volume', 'Volume publication year',
       'Evaluation year', 'Additional information', 'Causes_cancer'],
      dtype='object')

In [99]:
carcinogens.loc[carcinogens['Group']=='1'].head(1)

,CAS No.,Agent,Group,Volume,Volume publication year,Evaluation year,Additional information,Causes_cancer
68,NaN,Clonorchis sinensis (infection with),1,"61, 100B",2012,2009.0,NaN,Carcinogenic to humans


In [100]:
carcinogens=carcinogens.drop(columns=['CAS No.','Volume'])

In [101]:
carcinogens=carcinogens.rename(columns={'Agent':'chemical','Group':'group','Volume publication year':'publication_year','Evaluation year':'evaluation_year','Causes_cancer':'cancer_indicator'})

In [102]:
carcinogens=carcinogens.drop(columns=['Additional information'])

In [103]:
carcinogens['chemical']

0                     Alpha particles (see Radionuclides)
1                      Beta particles (see Radionuclides)
2       Boot and shoe manufacture and repair (see Leat...
3                             Chimney sweeping (see Soot)
4                   Chlorophenols (see Polychlorophenols)
                              ...                        
1115                           1,2-Diamino-4-nitrobenzene
1116                                2-Amino-4-nitrophenol
1117                              5-Nitro-ortho-anisidine
1118                        N-Methyl-N,4-dinitrosoaniline
1119                               tert-Amyl methyl ether
Name: chemical, Length: 1120, dtype: object

In [104]:
def clean_name(name):
    name = str(name).lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name


In [105]:
themaincosmetics['clean_chemical'] = themaincosmetics['chemical'].apply(clean_name)
carcinogens['clean_chemical'] = carcinogens['chemical'].apply(clean_name)

In [106]:
chemical_matches = {}
chemical_list = carcinogens['clean_chemical'].dropna().unique()

for chem in themaincosmetics['clean_chemical'].dropna().unique():
    match, score = process.extractOne(chem, chemical_list)
    if score >= 90:  
        chemical_matches[chem] = match

In [107]:
themaincosmetics['fuzzy_matched_chemical'] = themaincosmetics['clean_chemical'].map(chemical_matches)

In [108]:
merged = themaincosmetics.merge(
    carcinogens,
    left_on='fuzzy_matched_chemical',
    right_on='clean_chemical',
    how='left',
    
)

In [109]:
merged.columns

Index(['product', 'category', 'initial_reported_date',
       'most_recent_date_reported', 'discontinued_date',
       'chemical_created_date', 'chemical_updated_date', 'chemical_removed',
       'chemicalcount', 'titanium_dioxide', 'subcategory', 'brand', 'company',
       'chemical_x', 'discontinued_year', 'initial_reported_year',
       'most_recent_date_reported_year', 'chemical_created_year',
       'chemical_updated_year', 'chemical_removed_year', 'clean_chemical_x',
       'fuzzy_matched_chemical', 'chemical_y', 'group', 'publication_year',
       'evaluation_year', 'cancer_indicator', 'clean_chemical_y'],
      dtype='object')

In [110]:
overall_cosmetics=merged[['product','category','initial_reported_date','most_recent_date_reported','discontinued_date','chemical_created_date','chemical_updated_date','chemical_removed','chemicalcount','titanium_dioxide','subcategory','brand','company','discontinued_year','initial_reported_year','most_recent_date_reported_year','chemical_created_year','chemical_updated_year','chemical_removed_year','fuzzy_matched_chemical','group','publication_year','evaluation_year','cancer_indicator']]

In [111]:
subcategories=overall_cosmetics['subcategory'].value_counts().head().reset_index().subcategory

In [112]:
subcategories=overall_cosmetics.loc[overall_cosmetics.subcategory.isin(subcategories)]

In [113]:
subcategories=subcategories.groupby('subcategory')['fuzzy_matched_chemical'].value_counts().reset_index()

In [114]:
subcategories['rank']=subcategories.groupby('subcategory')['count'].rank(ascending=False)

In [115]:
subcategories=subcategories.rename(columns={'fuzzy_matched_chemical':'chemical','count':'product_count'})

In [116]:
products_titanium=overall_cosmetics.loc[overall_cosmetics['titanium_dioxide']==True]

In [117]:
company_titanium=products_titanium['company'].value_counts().reset_index()

In [118]:
company_titanium=company_titanium.rename(columns= {'count':'titanium_count'})

In [119]:
cosmeticscounts=overall_cosmetics['company'].value_counts().reset_index()
cosmeticscounts=cosmeticscounts.rename(columns={'count':'product_count'})

In [120]:
titanium_and_company=pd.merge(company_titanium,cosmeticscounts,on='company')

In [121]:
titanium_and_company.head(1)

,company,titanium_count,product_count
0,loreal,5743,5747


In [122]:
companiestitanium=titanium_and_company[titanium_and_company['product_count']>titanium_and_company['titanium_count']]

In [123]:
companiestitanium['percent_titanium_dioxide']= companiestitanium['titanium_count']/companiestitanium['product_count']
companiestitanium['percent_titanium_dioxide']=companiestitanium['percent_titanium_dioxide'].sort_values(ascending=False)

C:\Users\epoh2\AppData\Local\Temp\ipykernel_51696\1506024162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companiestitanium['percent_titanium_dioxide']= companiestitanium['titanium_count']/companiestitanium['product_count']
C:\Users\epoh2\AppData\Local\Temp\ipykernel_51696\1506024162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companiestitanium['percent_titanium_dioxide']=companiestitanium['percent_titanium_dioxide'].sort_values(ascending=False)


In [124]:
companiestitanium['percent_titanium_dioxide']=companiestitanium['percent_titanium_dioxide'].apply(lambda x: '{:.2%}'.format(x))

C:\Users\epoh2\AppData\Local\Temp\ipykernel_51696\4012231009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companiestitanium['percent_titanium_dioxide']=companiestitanium['percent_titanium_dioxide'].apply(lambda x: '{:.2%}'.format(x))


In [125]:
companiestitanium.head(1)

,company,titanium_count,product_count,percent_titanium_dioxide
0,loreal,5743,5747,99.93%


**Final cleaned data**

In [136]:
overall_cosmetics.to_csv('overall_cosmetics.csv')

In [138]:
companiestitanium.to_csv('companiestitanium.csv')

In [142]:
subcategories.to_csv('subcategories.csv')

**Junk**

In [133]:
the_main_cosmetics['company'].value_counts().head()

company
l'oreal            5747
s+                 5165
coty               5162
revlon             4341
bare essentials    3828
Name: count, dtype: int64

In [134]:
the_main_cosmetics.to_csv('the_main_cosmetics.csv')

In [135]:
products_titanium=the_main_cosmetics.loc[cosmetics3['Titanium_dioxide']==True]

NameError: name 'cosmetics3' is not defined

In [ ]:
company_titanium=products_titanium['company'].value_counts().reset_index()

In [ ]:
company_titanium=company_titanium.rename(columns= {'count':'titanium_count'})

In [ ]:
cosmeticscounts=the_main_cosmetics['company'].value_counts().reset_index()
cosmeticscounts=cosmeticscounts.rename(columns={'count':'product_count'})

In [ ]:
the_titanium_and_companies=pd.merge(company_titanium,cosmeticscounts,on='company')

In [ ]:
the_titanium_and_companies

In [ ]:
titanium_and_company=titanium_and_company[titanium_and_company['product_count']>titanium_and_company['titanium_count']]

In [ ]:
titanium_and_company['percent_titanium_dioxide']= titanium_and_company['titanium_count']/titanium_and_company['product_count']
titanium_and_company['percent_titanium_dioxide']=titanium_and_company['percent_titanium_dioxide'].sort_values(ascending=False)

In [ ]:
titanium_and_company['percent_titanium_dioxide']=titanium_and_company['percent_titanium_dioxide'].apply(lambda x: '{:.2%}'.format(x))

In [ ]:
titanium_and_company.sort_values('percent_titanium_dioxide',ascending=False).head(5)

In [ ]:
brand_per_company = the_main_cosmetics.groupby('company')['brand'].unique().reset_index()

In [ ]:
titanium_and_company = titanium_and_company.merge(brand_per_company, on='company', how='left')

In [ ]:
titanium_dioxide=titanium_and_company

In [ ]:
titanium_dioxide.to_csv('titanium_dioxide.csv')

In [ ]:
titanium_dioxide